In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
%matplotlib notebook
from Reinforcement_Learning import *
from Maze import *
from DrawMaze import *
import cv2
import matplotlib.pyplot as plt
import csv

In [2]:
PATH = 0 #道
WALL = 1 #壁
STR = 'S' #スタート
GOL = 'G' #ゴール
# mazeRead, mazeWrite, mazeDataFile = False, True, 'input/maze.csv'
mazeRead, mazeWrite, mazeDataFile = True, False, 'input/maze.csv'
if mazeRead:
    #迷路読み込み
    mazeData = []
    with open(mazeDataFile) as f:
        reader = csv.reader(f)
        for row in reader:
            mazeData.append(row)
    maze = Maze(mazeData, 1)
else:
    #迷路生成
    mWidth, mHeight = 14, 14
    maze = Maze([mWidth, mHeight], 0)
    maze.set_out_wall()
    maze.set_inner_wall_botaoshi()
    maze.set_start_goal()
    
maze.print_maze()
#迷路保存
if mazeWrite:
    maze.export_csv(mazeDataFile)
mWidth, mHeight = maze.get_width_height()
# 迷路データ maze.maze[x][y]

＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃
＃Ｓ　　　　　　　　　　　　＃　　　　　＃
＃　＃＃＃　＃＃＃＃＃＃＃＃＃　＃＃＃　＃
＃　＃　＃　　　　　＃　　　　　＃　＃　＃
＃　＃　＃＃＃＃＃　＃　＃　＃　＃　＃＃＃
＃　＃　　　＃　＃　＃　＃　＃　　　　　＃
＃　＃　＃　＃　＃　＃　＃＃＃　＃＃＃＃＃
＃　　　＃　＃　＃　　　　　＃　　　　　＃
＃＃＃　＃　＃　＃＃＃＃＃　＃＃＃　＃＃＃
＃　＃　＃　　　＃　　　＃　　　＃　＃　＃
＃　＃　＃　＃＃＃　＃＃＃　＃＃＃＃＃　＃
＃　　　＃　　　　　＃　＃　　　　　　　＃
＃＃＃　＃＃＃　＃＃＃　＃＃＃　＃＃＃＃＃
＃　　　＃　＃　＃　　　＃　＃　＃　　　＃
＃　＃＃＃　＃　＃　＃＃＃　＃　＃　＃　＃
＃　　　　　＃　　　　　＃　＃　＃　＃　＃
＃　＃　＃　＃＃＃＃＃　＃　＃　＃　＃　＃
＃　＃　＃　　　＃　＃　　　＃　　　＃　＃
＃　＃＃＃＃＃　＃　＃＃＃＃＃＃＃　＃＃＃
＃　　　　　＃　　　　　　　　　＃　　Ｇ＃
＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃＃


In [ ]:
# 強化学習
# q値更新
learningRate = 0.1 #学習係数α
discountRate = 0.9 #割引率γ
qUpdate = QLearning(learningRate, discountRate) # Q学習

#行動選択方法
ep = 0.3 #ε
actionSelect = EpGreedy(ep) #ε-greedy 

learning = ReinforcementLearning(mWidth, mHeight, qUpdate, actionSelect)

In [ ]:
#壁でないか確認
def is_not_wall(x, y):
    if maze.maze[x][y] == WALL:
        return False
    else:
        return True

In [ ]:
# 迷路描画機能
imgW = 400
imgH = 400
draw = DrawMaze(imgW, imgH, maze.maze)

In [ ]:
# 学習初期化
str_x, str_y = maze.get_start() #TODO Start位置の取得関数
gol_x, gol_y = maze.get_goal() #TODO Goal位置の取得関数
gol_reward = 30 #成功報酬
sx, sy = str_x, str_y
maxAction = 3000 # 最大行動数
maxResetAction = (mWidth + mHeight) * 30 # リセットになる行動回数
resetAction = 0

In [ ]:
for t in range(0, maxAction):
    # 次の行動を決定
    while True:
        nx, ny, action = learning.get_next_state(sx, sy)
        # 今回は次の行動が壁の時はキャンセルする
        if is_not_wall(nx, ny):
            break
    
    # Q値更新
    sq = learning.get_q_val(sx, sy, action)
    na, nq = learning.get_q_max_val(nx, ny)
    reward = gol_reward if (nx == gol_x and ny == gol_y) else 0
    p = [sq, nq, reward]
    learning.update_q_val(sx, sy, action, p)
    
    # 迷路描画
    draw.drawMaze(learning, gol_reward, sx, sy, nx, ny)
    sx, sy = nx, ny
    
    resetAction = resetAction + 1
    # リセット回数に達している、またはゴールならばスタートに戻る
    if (resetAction >= maxResetAction) or (sx == gol_x and sy == gol_y):
        resetAction = 0
        sx, sy = str_x, str_y    

In [ ]:
#結果出力(動画)
# encoder(for mp4)
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
# output file name, encoder, fps, size(fit to image size)
video = cv2.VideoWriter('video.mp4',fourcc, 120.0, (imgW, imgH))
if not video.isOpened():
    print("can't be opened")
    sys.exit()
    
for i in range(0, maxAction):
    # hoge0000.png, hoge0001.png,..., hoge0090.png
    img = cv2.imread('output/images/img_' + str(i)+ '.jpg')

    # can't read image, escape
    if img is None:
        print("can't read")
        break

    # add
    video.write(img)
    print("read image " + str(i))

video.release()
print('written')

read image 0
read image 1
read image 2
read image 3
read image 4
read image 5
read image 6
read image 7
read image 8
read image 9
read image 10
read image 11
read image 12
read image 13
read image 14
read image 15
read image 16
read image 17
read image 18
read image 19
read image 20
read image 21
read image 22
read image 23
read image 24
read image 25
read image 26
read image 27
read image 28
read image 29
read image 30
read image 31
read image 32
read image 33
read image 34
read image 35
read image 36
read image 37
read image 38
read image 39
read image 40
read image 41
read image 42
read image 43
read image 44
read image 45
read image 46
read image 47
read image 48
read image 49
read image 50
read image 51
read image 52
read image 53
read image 54
read image 55
read image 56
read image 57
read image 58
read image 59
read image 60
read image 61
read image 62
read image 63
read image 64
read image 65
read image 66
read image 67
read image 68
read image 69
read image 70
read image 71
re

read image 577
read image 578
read image 579
read image 580
read image 581
read image 582
read image 583
read image 584
read image 585
read image 586
read image 587
read image 588
read image 589
read image 590
read image 591
read image 592
read image 593
read image 594
read image 595
read image 596
read image 597
read image 598
read image 599
read image 600
read image 601
read image 602
read image 603
read image 604
read image 605
read image 606
read image 607
read image 608
read image 609
read image 610
read image 611
read image 612
read image 613
read image 614
read image 615
read image 616
read image 617
read image 618
read image 619
read image 620
read image 621
read image 622
read image 623
read image 624
read image 625
read image 626
read image 627
read image 628
read image 629
read image 630
read image 631
read image 632
read image 633
read image 634
read image 635
read image 636
read image 637
read image 638
read image 639
read image 640
read image 641
read image 642
read image

read image 1116
read image 1117
read image 1118
read image 1119
read image 1120
read image 1121
read image 1122
read image 1123
read image 1124
read image 1125
read image 1126
read image 1127
read image 1128
read image 1129
read image 1130
read image 1131
read image 1132
read image 1133
read image 1134
read image 1135
read image 1136
read image 1137
read image 1138
read image 1139
read image 1140
read image 1141
read image 1142
read image 1143
read image 1144
read image 1145
read image 1146
read image 1147
read image 1148
read image 1149
read image 1150
read image 1151
read image 1152
read image 1153
read image 1154
read image 1155
read image 1156
read image 1157
read image 1158
read image 1159
read image 1160
read image 1161
read image 1162
read image 1163
read image 1164
read image 1165
read image 1166
read image 1167
read image 1168
read image 1169
read image 1170
read image 1171
read image 1172
read image 1173
read image 1174
read image 1175
read image 1176
read image 1177
read ima

read image 1668
read image 1669
read image 1670
read image 1671
read image 1672
read image 1673
read image 1674
read image 1675
read image 1676
read image 1677
read image 1678
read image 1679
read image 1680
read image 1681
read image 1682
read image 1683
read image 1684
read image 1685
read image 1686
read image 1687
read image 1688
read image 1689
read image 1690
read image 1691
read image 1692
read image 1693
read image 1694
read image 1695
read image 1696
read image 1697
read image 1698
read image 1699
read image 1700
read image 1701
read image 1702
read image 1703
read image 1704
read image 1705
read image 1706
read image 1707
read image 1708
read image 1709
read image 1710
read image 1711
read image 1712
read image 1713
read image 1714
read image 1715
read image 1716
read image 1717
read image 1718
read image 1719
read image 1720
read image 1721
read image 1722
read image 1723
read image 1724
read image 1725
read image 1726
read image 1727
read image 1728
read image 1729
read ima

read image 2245
read image 2246
read image 2247
read image 2248
read image 2249
read image 2250
read image 2251
read image 2252
read image 2253
read image 2254
read image 2255
read image 2256
read image 2257
read image 2258
read image 2259
read image 2260
read image 2261
read image 2262
read image 2263
read image 2264
read image 2265
read image 2266
read image 2267
read image 2268
read image 2269
read image 2270
read image 2271
read image 2272
read image 2273
read image 2274
read image 2275
read image 2276
read image 2277
read image 2278
read image 2279
read image 2280
read image 2281
read image 2282
read image 2283
read image 2284
read image 2285
read image 2286
read image 2287
read image 2288
read image 2289
read image 2290
read image 2291
read image 2292
read image 2293
read image 2294
read image 2295
read image 2296
read image 2297
read image 2298
read image 2299
read image 2300
read image 2301
read image 2302
read image 2303
read image 2304
read image 2305
read image 2306
read ima

read image 2823
read image 2824
read image 2825
read image 2826
read image 2827
read image 2828
read image 2829
read image 2830
read image 2831
read image 2832
read image 2833
read image 2834
read image 2835
read image 2836
read image 2837
read image 2838
read image 2839
read image 2840
read image 2841
read image 2842
read image 2843
read image 2844
read image 2845
read image 2846
read image 2847
read image 2848
read image 2849
read image 2850
read image 2851
read image 2852
read image 2853
read image 2854
read image 2855
read image 2856
read image 2857
read image 2858
read image 2859
read image 2860
read image 2861
read image 2862
read image 2863
read image 2864
read image 2865
read image 2866
read image 2867
read image 2868
read image 2869
read image 2870
read image 2871
read image 2872
read image 2873
read image 2874
read image 2875
read image 2876
read image 2877
read image 2878
read image 2879
read image 2880
read image 2881
read image 2882
read image 2883
read image 2884
read ima